In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import pandas as pd
from random import uniform

# WebDriver 설정
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')  # Headless 모드
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=chrome_options)

def get_keyword(text):
    return text.replace(" ", "%20")

def sort_kind(index):
    if index == 1:
        return 'vcount'
    elif index == 2:
        return 'date'
    else:
        return 'none'

keyword = '강아지'  # 크롤링할 키워드 설정
driver.get('https://kin.naver.com/search/list.nhn?query=' + get_keyword(keyword))
time.sleep(uniform(0.1, 1.0))

page_index = 1
f = '2020.07.11'  # 크롤링 시작 일자
t = '2024.09.10'  # 크롤링 종료 일자
period_txt = "&period=" + f + ".%7C" + t + "."
_sort_kind = sort_kind(2)

page_url = []

# 최대 페이지 수 설정
max_pages = 1

while page_index <= max_pages:
    time.sleep(uniform(0.1, 1.0))
    driver.get('https://kin.naver.com/search/list.nhn?' + "&sort=" + _sort_kind + '&query=' + get_keyword(keyword) + period_txt + "&section=kin" + "&page=" + str(page_index))
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    tags = soup.find_all('a', class_="_searchListTitleAnchor")
    for tag in tags:
        url = tag['href']
        page_url.append(url)

    page_index += 1  # 페이지 증가

data = []

for url in page_url:
    driver.get(url)
    time.sleep(uniform(0.5, 1.0))  # 페이지 로딩 시간을 위한 대기
    try:
        # 제목 가져오기 (div 계층을 사용한 접근)
        title = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/div[1]/div[1]/div[1]'))
        ).text
    except:
        title = ""

    # "강아지" 키워드가 제목에 포함되어 있는지 확인
    if keyword in title:
        try:
            # 질문 가져오기 (div 계층을 사용한 접근)
            question_txt = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/div[1]/div[1]/div[3]'))
            ).text
        except:
            question_txt = ""
        
        # 답변 추출
        answers = driver.find_elements(By.CSS_SELECTOR, ".se-main-container .se-component-content")
        answer_text = [answer.text for answer in answers]
        
        if not answer_text:  # 답변이 없는 경우도 처리
            answer_text = [""]

        # 작성일 가져오기 (XPath 사용)
        try:
            date = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/div[1]/div[1]/div[2]/span[3]'))
            ).text
        except:
            date = "날짜 없음"

        for answer in answer_text:
            data.append([title, question_txt, answer, date])

# 데이터프레임에 날짜 컬럼 추가
dog_df = pd.DataFrame(data, columns=['제목', '질문', '답변', '날짜'])
dog_df

# 필요에 따라 데이터를 파일로 저장
# dog_df.to_excel('naver_kin_crawling_result.xlsx', index=False)

driver.quit()


In [9]:
dog_df

,제목,질문,답변,날짜
0,질문\n강아지 다리 쪽 털빠짐,제가 요즘 바빠서 강아지 신경을 잘 못 쓰다가 최근에 강아지 다리 쪽 털이 빠져서 ...,"안녕하세요, 반려동물 건강관리 전문약사 우성민입니다.\n강아지 다리 쪽 털빠짐에 대...",작성일\n2024.09.10
1,질문\n강아지 다리 쪽 털빠짐,제가 요즘 바빠서 강아지 신경을 잘 못 쓰다가 최근에 강아지 다리 쪽 털이 빠져서 ...,원형으로 빠지는게 아니라 링웜종류는 아닐것 같고\n피부문제가 아니라 호르몬 문제일 ...,작성일\n2024.09.10
2,질문\n강아지 다리 쪽 털빠짐,제가 요즘 바빠서 강아지 신경을 잘 못 쓰다가 최근에 강아지 다리 쪽 털이 빠져서 ...,탈모 원인은 다양해서 한가지 딱 말씀드리긴 어렵구요\n저희 강아지는 병원 안가고도 ...,작성일\n2024.09.10
3,질문\n스피츠 성견 강아지 산책 짖음 어떻게 해야하나요,"스피츠 성견 강아지인데요, 산책할 때마다 짖는게 조금 심합니다\n\n산책할 때 사람...",예.\n반려견의 성격에 따라서 짖음이 있을수가 있습니다.\n짖는다고 산책을 하지 않...,작성일\n2024.09.10
4,질문\n스피츠 성견 강아지 산책 짖음 어떻게 해야하나요,"스피츠 성견 강아지인데요, 산책할 때마다 짖는게 조금 심합니다\n\n산책할 때 사람...",사회성이 부족하기 때문입니다.\n안녕하세요 저는 동물 심리 전문가이며 애견 훈련...,작성일\n2024.09.10
5,질문\n스피츠 성견 강아지 산책 짖음 어떻게 해야하나요,"스피츠 성견 강아지인데요, 산책할 때마다 짖는게 조금 심합니다\n\n산책할 때 사람...",존재하지 않는 이미지입니다.,작성일\n2024.09.10
6,질문\n스피츠 성견 강아지 산책 짖음 어떻게 해야하나요,"스피츠 성견 강아지인데요, 산책할 때마다 짖는게 조금 심합니다\n\n산책할 때 사람...",☞ 책 소...,작성일\n2024.09.10
7,질문\n스피츠 성견 강아지 산책 짖음 어떻게 해야하나요,"스피츠 성견 강아지인데요, 산책할 때마다 짖는게 조금 심합니다\n\n산책할 때 사람...",짖음은 반가워서 그러는 경우도 있고\n무서우니까 다가오지 말라고 경고성으로 짖는 경...,작성일\n2024.09.10
8,질문\n스피츠 성견 강아지 산책 짖음 어떻게 해야하나요,"스피츠 성견 강아지인데요, 산책할 때마다 짖는게 조금 심합니다\n\n산책할 때 사람...",이건 정확하게 어떻게 표현해야할지를 모르는 상태입니다.\n그중에서 강아지가 가장 선...,작성일\n2024.09.10
9,질문\n강아지가 너무 소중해요ㅠㅠ,제가 강아지 키우는걸 4년동안 쫄라서 키우게 됐는데 하루하루 지날수록 더 소중하고 ...,그럼 부모님하고는 어떻게 같이 삽니까.. 부모님도 언젠간 돌아가실텐데요. \n그냥 ...,작성일\n2024.09.10
